# Azure ML Workspace

## Importing libraries

In [ ]:
from azureml.core import Workspace, Environment, Model
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
import json
import pandas as pd

## Initialise workspace
Initialise a workspace using the values from config.json
If this file is not found, an exception occurs
When an exception is detected, a workspace using the defined details is created

In [ ]:
try:
    ## Try to load workspace
    wspace = Workspace.from_config()
except Exception as ex:
    ## Create new workspace if existing is not found
    wspace = Workspace.create(name="classification-workspace",
                                  subscription_id="11636c4c-f408-4202-980a-b738e97f4d7f",
                                  resource_group="classification-resources",
                                  create_resource_group=True,
                                  location="uksouth")

    ## Write workspace config to a local file
    wspace.write_config()

print("Workspace successfully loaded")

## Create an environment
Create an environment for the model to run on the Azure servers

In [ ]:
##  Load conda dependencies required for this environment.
##  Register environment for the workspace
env = Environment("env")

conda_dep = CondaDependencies.create(pip_packages=["pandas == 1.1.3",
                                                   "azureml-core",
                                                   "azureml-defaults >= 1.0.45"],
                                     conda_packages=["scikit-learn == 0.23.2"])

env.python.conda_dependencies = conda_dep

env.register(workspace=wspace)

print("Environment Registered")

## Get the environment created from the workspace and create a file to verify it has been defined
my_env = Environment.get(workspace=wspace, name="env")
my_env.save_to_directory("./environ", overwrite=True)

print("Environment Saved")

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=my_env)

## Register the models
Register the saved model created by "knn-model-tuning" and "svm-model-tuning" for use on Azure ML

In [ ]:
knn_model = Model.register(workspace=wspace,
                       model_name="knn",
                       model_path="models/knn.pkl",
                       model_framework="ScikitLearn",
                       description="Tuned KNN for Genre Classification")

svm_model = Model.register(workspace=wspace,
                       model_name="svm",
                       model_path="models/svm.pkl",
                       model_framework="ScikitLearn",
                       description="Tuned SVM for Genre Classification")

## Set up Azure Container Instance
Define ACI Webservice

In [ ]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

## Deploy the models
Deploy the models onto the defined ACI container using the set environment, inference config and workspace

In [ ]:
service = Model.deploy(workspace=wspace,
                       name="music-genre-service",
                       models=[knn_model, svm_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.get_logs())

print(service.scoring_uri)

## Test the web service
Test the web service has successfully deployed using test data

In [ ]:
# Reading the test set from file
data_df = pd.read_csv("testing_data.csv")
data_json_str = data_df.to_json(orient="records")

response_pred, response_acc, response_auroc, response_f1 = service.run(json.dumps(data_json_str))

response_pred_df = pd.DataFrame(response_pred)

print(response_pred_df)
print(response_acc)
print(response_auroc)
print(response_f1)

## Delete ACI
Delete the ACI instance to clean up
(This will also delete the endpoint)

In [ ]:
# service.delete()